In [1]:
import pysam
import urllib.request
import traceback
import pysam
from pyspark import SparkConf, SparkContext

conf = (SparkConf().setMaster("local[*]").setAppName("My app"))
sc = SparkContext(conf=conf)


def find_kmers(string, k):
      kmers = []
      n = len(string)
      for i in range(0, n-k+1):
           kmers.append(string[i:i+k])
      return kmers

base_url = "http://130.238.29.253:8080/swift/v1/1000-genomes-dataset"
files = urllib.request.urlopen(base_url).read().split()
bams = [s.strip() for s in files if "bai" not in str(s) and "bas" not in str(s)]

In [2]:
sequences = []

for i in range(2):
    bam_url = base_url + "/" + str(bams[i], 'utf-8').replace("'","")
    try:
        with pysam.AlignmentFile(bam_url, "rb") as bam:
            for b in bam.fetch(until_eof=True):
                if b.is_unmapped:
                    sequences.append(b.query_alignment_sequence)
                
    except Exception:
        print(Exception)
        traceback.print_exc()

In [ ]:
kmers = []

for s in sequences:
    kmers = kmers + find_kmers(s, 10)


In [ ]:
with open('output.txt', 'w') as f:
    for k in kmers:
        f.write(k + '\n')

In [ ]:
text_file = sc.textFile("output.txt")

counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

counts.saveAsTextFile("reduced.txt")


In [ ]:
sc.stop()